## Rough Draft, Still Need to Check Code

These are the needed libraries to be imported as well as the boilerplate functions

In [101]:
import pandas as pd
import numpy as np
import nltk
import math
from sklearn.svm import LinearSVC
from sklearn.svm import OneClassSVM
from collections import defaultdict


df_part1 = pd.read_csv('raw/data_part1.csv')
df_part2 = pd.read_csv('raw/data_part2.csv')
df_part3 = pd.read_csv('raw/data_part3.csv')
df_part4 = pd.read_csv('raw/data_part4.csv')
df_part5 = pd.read_csv('raw/data_part5.csv')

def get_orig_rows(author):
    rows = []
    for i,row in df_part1[df_part1['creator'] == author].iterrows():
        rows.append([list(row)[0]])
    return rows

UNK = "!!!<UNK>!!!"

def preprocess(text):
    tokens = nltk.tokenize.casual_tokenize(text, preserve_case=False)
    return [t for t in tokens if t.isalpha()]

def make_index(tokens):
    tokens.append(UNK)
    index = {}
    max_index = 0
    for token in tokens:
        if token not in index:
            index[token] = max_index
            max_index += 1
    return index

def embed_bow(tokens, index):
    a = np.zeros(len(index))
    for token in tokens:
        if token in index:
            a[index[token]] += 1
        else:
            a[index[UNK]] += 1
    return a

def reverse_index(index):
    return [w for i, w in sorted(map(tuple, map(reversed, index.items())))]


def one_row_per_embedding_bow(df, index):
    rows = []
    for i, row in df.iterrows():
        word_vec = embed_bow(preprocess(row['sample']), index)
        new_row = list(word_vec) 
        rows.append(new_row)
    bow_df = pd.DataFrame(rows)
    bow_df.columns = reverse_index(index)
    return bow_df

def embed_bow_freq(tokens, all_tokens, index):
    a = np.zeros(len(index))
    for token in set(tokens):
        if token in index:
            local_freq = tokens.count(token) / len(tokens)
            global_freq = all_tokens.count(token) / len(all_tokens)
            a[index[token]] = math.log(local_freq / (global_freq+1))
        """ 
        else:
            local_freq = tokens.count(token) / len(tokens)
            global_freq = all_tokens.count(token) / len(all_tokens)
            a[index[UNK]] = math.log(local_freq / (global_freq+1))
        """
    return a

def one_row_per_embedding_bow_freq(df, all_tokens, index):
    rows = []
    for i, row in df.iterrows():
        word_vec = embed_bow_freq(preprocess(row['sample']), all_tokens, index)
        new_row = list(word_vec) 
        rows.append(new_row)
    bow_freq_df = pd.DataFrame(rows)
    bow_freq_df.columns = reverse_index(index)
    return bow_freq_df

def precision(df, target, value):
    predicted_value = df[df['!predicted'] == value]
    actually_value = df[df[target] == value]
    predicted_and_actually_value = actually_value[actually_value['!predicted'] == value]
    
    return len(predicted_and_actually_value)/len(predicted_value)

def recall(df, target, value):
    predicted_value = df[df['!predicted'] == value]
    actually_value = df[df[target] == value]
    predicted_and_actually_value = actually_value[actually_value['!predicted'] == value]
    return len(predicted_and_actually_value)/len(actually_value)

def rand_index(df, value):
    num_val = len(df[df["!predicted"] == value])
    return num_val / len(df)

This function gets all imitations and identifies the intended athor as well as the original imitator. 

In [4]:
imitator_count = 0

def get_imitations(df):
    global imitator_count
    rows = []
    for i, row in df.iterrows():
        if 'participant' in list(row)[0]:
            imitator_count += 1
        else:
            if type(list(row)[1]) != float and list(row)[1] != ' ':
                text = list(row)[1]
                imitator = 'imitator'+str(imitator_count)
                if 'p1' in list(row)[2]:
                    int_author = 'participant 1'
                elif 'p2' in list(row)[2]:
                    int_author = 'participant 2'
                elif 'p3' in list(row)[2]:
                    int_author = 'participant 3'   
                msg_type = list(row)[3]
                row = [text, imitator, int_author, msg_type]
                rows.append(row)
    return rows
                
imitation_rows = get_imitations(df_part2) + get_imitations(df_part3) + get_imitations(df_part4)
df_imit =  pd.DataFrame(imitation_rows)
df_imit.columns = ['sample', 'imitator', 'intended author', 'type']

This block creates a dataframe for all imitator samples 

In [5]:
imitator_sample_count = 0

def get_imitation_samples(df):
    global imitator_sample_count
    rows = []
    for i, row in df.iterrows():
        if 'participant' in list(row)[0]:
            imitator_sample_count += 1
        else:
            text = list(row)[0]
            imitator = 'imitator'+str(imitator_sample_count)   
            msg_type = list(row)[3]
            row = [text, imitator, msg_type]
            rows.append(row)
    return rows

imit_sample_rows = get_imitation_samples(df_part2) + get_imitation_samples(df_part3) + get_imitation_samples(df_part4)
df_imit_samples = pd.DataFrame(imit_sample_rows)
df_imit_samples.columns = ['sample', 'imitator', 'type']

More boilerplate functions for imitators samples and for imitations themselves. 

In [85]:
def get_imitator_sample_rows(df, imitator):
    rows = []
    for i,row in df[df['imitator'] == imitator].iterrows():
        sample = list(row)[0]
        rows.append([sample])
    return rows

def get_orig_rows_with_type(df, author, msgType):
    rows = []
    for i,row in df[df['creator'] == author].iterrows():
        if list(row)[2] == msgType:
            rows.append([list(row)[0], 'different'])
    return rows

def get_imitation_test_set(df, imitator, intended_author):
    rows = []
    for i,row in df.iterrows():
        r = list(row)
        if r[2] == intended_author and r[1] == imitator:
            rows.append([r[0], "same"])
    df_final = pd.DataFrame(rows)
    return df_final

def get_attacks_on_author(df, intended_author):
    rows = []
    for i,row in df.iterrows():
        r = list(row)
        if r[2] == intended_author:
            rows.append([r[0]])
    return rows

# Baseline

Here, we will create the training set for the baseline, namely the first two authors and their respective imitators. 

In [25]:
auth1_auth2_rows = get_orig_rows("participant 1") + get_orig_rows("participant 2")
auth1_auth2_imit_rows = get_attacks_on_author(df_imit, "participant 1") + get_attacks_on_author(df_imit, "participant 2")
baseline_df_train = pd.DataFrame(auth1_auth2_rows + auth1_auth2_imit_rows).sample(frac=1)
baseline_df_train.columns = ['sample']

Now, we create the test set, namely the originaal message and attempted imitations of author 3. 

In [26]:
auth3_rows = get_orig_rows("participant 3")
auth3_imit_rows = get_attacks_on_author(df_imit, "participant 3")
baseline_df_test = pd.DataFrame(auth3_rows + auth3_imit_rows).sample(frac=1)
baseline_df_test.columns = ['sample']

We then create bag of words embedding for the training and test sets.

In [32]:
all_train = " ".join(baseline_df_train['sample'])
all_train_tokens = preprocess(all_train)
train_index = make_index(all_train_tokens)
all_train_cols = reverse_index(train_index)

df_train_bow = one_row_per_embedding_bow(baseline_df_train, train_index)
df_test_bow = one_row_per_embedding_bow(baseline_df_test, train_index)

df_train_bow["!actual"] = [1 for i in range(len(df_train_bow))]
df_test_bow["!actual"] = [-1 for i in range(len(df_test_bow))]

Now we train the One Class SVM and evaluate thtraining and test sets. 

In [48]:
ocsvm_train_bow = df_train_bow.copy()
ocsvm_test_bow = df_test_bow.copy()

baseline_ocsvm = OneClassSVM()
baseline_ocsvm.fit(ocsvm_train_bow[all_train_cols])

ocsvm_train_bow['!predicted'] = baseline_ocsvm.predict(ocsvm_train_bow[all_train_cols])
ocsvm_test_bow['!predicted'] = baseline_ocsvm.predict(ocsvm_test_bow[all_train_cols])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [51]:
print("Known Authors P/R")
print("Precision: " + str(precision(ocsvm_train_bow, "!actual", 1)))
print("Recall: " + str(recall(ocsvm_train_bow, "!actual", 1)))
print("Rand Index: " + str(rand_index(ocsvm_train_bow, 1)))

print()

print("Unknown Authors P/R")
print("Precision: " + str(precision(ocsvm_test_bow, "!actual", -1)))
print("Recall: " + str(recall(ocsvm_test_bow, "!actual", -1)))
print("Rand Index: " + str(rand_index(ocsvm_test_bow, -1)))

Known Authors P/R
Precision: 1.0
Recall: 0.5076923076923077
Rand Index: 0.5076923076923077

Unknown Authors P/R
Precision: 1.0
Recall: 0.5319148936170213
Rand Index: 0.5319148936170213


# Baseline Pt2

Now we perform the same baseline test but on a different embedding (relative frequency ratios on a log scaling). 

In [81]:
freq_train_bow = one_row_per_embedding_bow_freq(baseline_df_train, all_train_tokens, train_index)
freq_test_bow = one_row_per_embedding_bow_freq(baseline_df_test, all_train_tokens, train_index)

freq_train_bow["!actual"] = [1 for i in range(len(freq_train_bow))]
freq_test_bow["!actual"] = [1 for i in range(len(freq_test_bow))]

In [82]:
freq_ocsvm_train_bow = freq_train_bow.copy()
freq_ocsvm_test_bow = freq_test_bow.copy()

freq_baseline_ocsvm = OneClassSVM()
freq_baseline_ocsvm.fit(freq_ocsvm_train_bow[all_train_cols])

freq_ocsvm_train_bow['!predicted'] = \
freq_baseline_ocsvm.predict(freq_ocsvm_train_bow[all_train_cols])

freq_ocsvm_test_bow['!predicted'] = \
freq_baseline_ocsvm.predict(freq_ocsvm_test_bow[all_train_cols])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [83]:
print("Known Authors P/R")
print("Precision: " + str(precision(freq_ocsvm_train_bow, "!actual", 1)))
print("Recall: " + str(recall(freq_ocsvm_train_bow, "!actual", 1)))
print("Rand Index: " + str(rand_index(freq_ocsvm_train_bow, 1)))

print()

print("Unknown Authors P/R")
print("Precision: " + str(precision(freq_ocsvm_test_bow, "!actual", 1)))
print("Recall: " + str(recall(freq_ocsvm_test_bow, "!actual", 1)))
print("Rand Index: " + str(rand_index(freq_ocsvm_test_bow, 1)))

Known Authors P/R
Precision: 1.0
Recall: 0.5076923076923077
Rand Index: 0.5076923076923077

Unknown Authors P/R
Precision: 1.0
Recall: 0.925531914893617
Rand Index: 0.925531914893617


# Comparison Using SVM

Here, we will first preprocess each author and imitator's reference sets and store them into a dictionary. 

In [71]:
author_dict = defaultdict(list)
imitator_dict = defaultdict(list)

for i,row in df_part1.iterrows():
    r = list(row)
    author_dict[r[1]] += preprocess(r[0])

for i,row in df_imit_samples.iterrows():
    r = list(row)
    imitator_dict[r[1]] += preprocess(r[0])
    

In [107]:
df_imit_comp = df_imit.copy()
svm1_distance = []
svm2_distance = []

for i,row in df_imit_comp.iterrows():
    r = list(row)
    svm1 = OneClassSVM()
    svm2 = OneClassSVM()
    msg_tokens = preprocess(r[0])
    
    messages = get_orig_rows(r[2]) + \
    get_imitator_sample_rows(df_imit_samples, r[1])
    
    df_messages = pd.DataFrame(messages).sample(frac=1)
    df_messages.columns = ['sample']
    
    all_msg = " ".join(df_messages['sample'])
    all_msg_tokens = preprocess(all_msg)
    msg_index = make_index(all_msg_tokens)
    
    df_train_svm1 = one_row_per_embedding_bow_freq(df_messages, author_dict[r[2]], msg_index)
    df_train_svm2 = one_row_per_embedding_bow_freq(df_messages, imitator_dict[r[1]], msg_index)
    
    svm1.fit(df_train_svm1[msg_index])
    svm2.fit(df_train_svm2[msg_index])
    
    #print(svm1.decision_function(embed_bow_freq(msg_tokens, author_dict[r[2]], msg_index)))
    
    svm1_distance.append(svm1.decision_function(embed_bow_freq(msg_tokens, author_dict[r[2]], msg_index).reshape(1, -1)))
    svm2_distance.append(svm2.decision_function(embed_bow_freq(msg_tokens, imitator_dict[r[1]], msg_index).reshape(1, -1)))
    
df_imit_comp["SVM1_dist"] = svm1_distance
df_imit_comp["SVM2_dist"] = svm2_distance
    

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

In [108]:
df_imit_comp

,sample,imitator,intended author,type,SVM1_dist,SVM2_dist
0,"Wanda, Can you believe winter is finally here?...",imitator1,participant 1,email,[-0.6173709809779032],[-0.6171658364957338]
1,The only way of finding out why your partner d...,imitator2,participant 1,email,[-0.4238050484457261],[-0.42463692177045353]
2,I love reading murder mysteries and your lates...,imitator3,participant 3,email,[-0.5300859048959089],[-0.5307132779061514]
3,there was a little tussle between spring and f...,imitator4,participant 2,email,[-0.48186005052936753],[-0.48183797294263575]
4,I'd like a barbecue with my family and some fr...,imitator5,participant 3,email,[-0.3488404295536235],[-0.34864137582739285]
5,Hi Sarah. Congratulations on becoming Presiden...,imitator6,participant 1,email,[-0.017253697286235603],[-0.017773869304679613]
6,Hi Emily. Congratulations on this great milest...,imitator7,participant 1,email,[-0.21068437585831878],[-0.21088472741194786]
7,"You know, I would love to have a barbecue with...",imitator8,participant 3,email,[-0.16873113099501946],[-0.16830747406942836]
8,Hi Kaitlyn. I cannot believe it is already Jun...,imitator9,participant 1,email,[-0.10799637765284453],[-0.10772205302029514]
9,Hey Cam. I can't wait until summer arrives. It...,imitator10,participant 1,email,[-0.2877056415828301],[-0.2872008887877182]
